# Load model selection table
This utility function generates model selection tuples (model architecture, compile parameters, fit parameters) for both hyper-parameter search and model architecture search.  The model selection table and associated summary table are used by the multiple model fit feature of MADlib.  

This utility was added in MADlib 1.17.

## Table of contents

<a href="#define_model_arch">1. Define model architecture table</a>

<a href="#load_model_arch">2. Load model architecture</a>

<a href="#load_model_selection">3. Load model selection table</a>

<a href="#load_model_selection_manual">4. Create model selection table manually</a>

<a href="#load_model_selection_auto">5. Generate hyperparameters automatically</a>

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - direct external IP access
#%sql postgresql://gpadmin@34.67.65.96:5432/madlib

# Greenplum Database 5.x on GCP - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [17]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.17-dev, git revision: rel/v1.16-54-gec5614f, cmake configuration time: Wed Dec 18 17:08:05 UTC 2019, build type: release, build system: Linux-3.10.0-1062.4.3.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


<a id="define_model_arch"></a>
# 1. Define model architecture table
The model selection loader works in conjunction with the model architecture table, so we first create a model architecture table with two different models.  See http://madlib.apache.org/docs/latest/group__grp__keras__model__arch.html for more details on the model architecture table.

Import Keras libraries

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Define model architecture with 1 hidden layer:

In [19]:
model1 = Sequential()
model1.add(Dense(10, activation='relu', input_shape=(4,)))
model1.add(Dense(10, activation='relu'))
model1.add(Dense(3, activation='softmax'))
    
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_9 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [20]:
model1.to_json()

'{"class_name": "Sequential", "keras_version": "2.1.6", "config": [{"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_8", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "batch_input_shape": [null, 4], "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_9", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10,

'{"class_name": "Sequential", "keras_version": "2.1.6", "config": [{"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_1", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "linear", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 2, "batch_input_shape": [null, 3], "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "new_dense", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "linear", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "use_bias": true, "activity_regularizer": null}}], "backend": "tensorflow"}'
        

Define model architecture with 2 hidden layers:

In [21]:
model2 = Sequential()
model2.add(Dense(10, activation='relu', input_shape=(4,)))
model2.add(Dense(10, activation='relu'))
model2.add(Dense(10, activation='relu'))
model2.add(Dense(3, activation='softmax'))
    
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_12 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_13 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 33        
Total params: 303
Trainable params: 303
Non-trainable params: 0
_________________________________________________________________


In [22]:
model2.to_json()

'{"class_name": "Sequential", "keras_version": "2.1.6", "config": [{"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_11", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "batch_input_shape": [null, 4], "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_12", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 1

'{"class_name": "Sequential", "keras_version": "2.1.6", "config": [{"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_4", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "batch_input_shape": [null, 4], "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_5", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_6", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_7", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "softmax", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 3, "use_bias": true, "activity_regularizer": null}}], "backend": "tensorflow"}'

<a id="load_model_arch"></a>
# 2. Load model architecture

Load both into model architecture table:

In [23]:
%%sql
DROP TABLE IF EXISTS model_arch_library;

SELECT madlib.load_keras_model('model_arch_library',  -- Output table,
                               
$$
{"class_name": "Sequential", "keras_version": "2.1.6", "config": [{"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_1", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "batch_input_shape": [null, 4], "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_2", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_3", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "softmax", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 3, "use_bias": true, "activity_regularizer": null}}], "backend": "tensorflow"}
$$
::json,         -- JSON blob
                               NULL,                  -- Weights
                               'Sophie',              -- Name
                               'MLP with 1 hidden layer'       -- Descr
);

SELECT madlib.load_keras_model('model_arch_library',  -- Output table,
                               
$$
{"class_name": "Sequential", "keras_version": "2.1.6", "config": [{"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_4", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "dtype": "float32", "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "batch_input_shape": [null, 4], "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_5", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_6", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "relu", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 10, "use_bias": true, "activity_regularizer": null}}, {"class_name": "Dense", "config": {"kernel_initializer": {"class_name": "VarianceScaling", "config": {"distribution": "uniform", "scale": 1.0, "seed": null, "mode": "fan_avg"}}, "name": "dense_7", "kernel_constraint": null, "bias_regularizer": null, "bias_constraint": null, "activation": "softmax", "trainable": true, "kernel_regularizer": null, "bias_initializer": {"class_name": "Zeros", "config": {}}, "units": 3, "use_bias": true, "activity_regularizer": null}}], "backend": "tensorflow"}
$$
::json,         -- JSON blob
                               NULL,                  -- Weights
                               'Maria',               -- Name
                               'MLP with 2 hidden layers'       -- Descr
);

SELECT * FROM model_arch_library ORDER BY model_id;

Done.
1 rows affected.
1 rows affected.
2 rows affected.


model_id,model_arch,model_weights,name,description,__internal_madlib_id__
1,"{u'class_name': u'Sequential', u'keras_version': u'2.1.6', u'config': [{u'class_name': u'Dense', u'config': {u'kernel_initializer': {u'class_name': u'VarianceScaling', u'config': {u'distribution': u'uniform', u'scale': 1.0, u'seed': None, u'mode': u'fan_avg'}}, u'name': u'dense_1', u'kernel_constraint': None, u'bias_regularizer': None, u'bias_constraint': None, u'dtype': u'float32', u'activation': u'relu', u'trainable': True, u'kernel_regularizer': None, u'bias_initializer': {u'class_name': u'Zeros', u'config': {}}, u'units': 10, u'batch_input_shape': [None, 4], u'use_bias': True, u'activity_regularizer': None}}, {u'class_name': u'Dense', u'config': {u'kernel_initializer': {u'class_name': u'VarianceScaling', u'config': {u'distribution': u'uniform', u'scale': 1.0, u'seed': None, u'mode': u'fan_avg'}}, u'name': u'dense_2', u'kernel_constraint': None, u'bias_regularizer': None, u'bias_constraint': None, u'activation': u'relu', u'trainable': True, u'kernel_regularizer': None, u'bias_initializer': {u'class_name': u'Zeros', u'config': {}}, u'units': 10, u'use_bias': True, u'activity_regularizer': None}}, {u'class_name': u'Dense', u'config': {u'kernel_initializer': {u'class_name': u'VarianceScaling', u'config': {u'distribution': u'uniform', u'scale': 1.0, u'seed': None, u'mode': u'fan_avg'}}, u'name': u'dense_3', u'kernel_constraint': None, u'bias_regularizer': None, u'bias_constraint': None, u'activation': u'softmax', u'trainable': True, u'kernel_regularizer': None, u'bias_initializer': {u'class_name': u'Zeros', u'config': {}}, u'units': 3, u'use_bias': True, u'activity_regularizer': None}}], u'backend': u'tensorflow'}",None,Sophie,MLP with 1 hidden layer,__madlib_temp_80732521_1576707528_41018934__
2,"{u'class_name': u'Sequential', u'keras_version': u'2.1.6', u'config': [{u'class_name': u'Dense', u'config': {u'kernel_initializer': {u'class_name': u'VarianceScaling', u'config': {u'distribution': u'uniform', u'scale': 1.0, u'seed': None, u'mode': u'fan_avg'}}, u'name': u'dense_4', u'kernel_constraint': None, u'bias_regularizer': None, u'bias_constraint': None, u'dtype': u'float32', u'activation': u'relu', u'trainable': True, u'kernel_regularizer': None, u'bias_initializer': {u'class_name': u'Zeros', u'config': {}}, u'units': 10, u'batch_input_shape': [None, 4], u'use_bias': True, u'activity_regularizer': None}}, {u'class_name': u'Dense', u'config': {u'kernel_initializer': {u'class_name': u'VarianceScaling', u'config': {u'distribution': u'uniform', u'scale': 1.0, u'seed': None, u'mode': u'fan_avg'}}, u'name': u'dense_5', u'kernel_constraint': None, u'bias_regularizer': None, u'bias_constraint': None, u'activation': u'relu', u'trainable': True, u'kernel_regularizer': None, u'bias_initializer': {u'class_name': u'Zeros', u'config': {}}, u'units': 10, u'use_bias': True, u'activity_regularizer': None}}, {u'class_name': u'Dense', u'config': {u'kernel_initializer': {u'class_name': u'VarianceScaling', u'config': {u'distribution': u'uniform', u'scale': 1.0, u'seed': None, u'mode': u'fan_avg'}}, u'name': u'dense_6', u'kernel_constraint': None, u'bias_regularizer': None, u'bias_constraint': None, u'activation': u'relu', u'trainable': True, u'kernel_regularizer': None, u'bias_initializer': {u'class_name': u'Zeros', u'config': {}}, u'units': 10, u'use_bias': True, u'activity_regularizer': None}}, {u'class_name': u'Dense', u'config': {u'kernel_initializer': {u'class_name': u'VarianceScaling', u'config': {u'distribution': u'uniform', u'scale': 1.0, u'seed': None, u'mode': u'fan_avg'}}, u'name': u'dense_7', u'kernel_constraint': None, u'bias_regularizer': None, u'bias_constraint': None, u'activation': u'softmax', u'trainable': True, u'kernel_regularizer': None, u'bias_initializer': {u'class_name': u'Zeros', u'config': {}}, u'units': 3, u'use_bias': True, u'activity_regularizer': None}}], u'backend': u'tensorflow'}",None,Maria,MLP with 2 hidden layers,__madlib_temp_54900499_1

<a id="load_model_selection"></a>
# 3.  Load model selection table

Select the model(s) from the model architecture table that you want to run, along with the compile and fit parameters.  Unique combinations will be created for the set of model selection parameters:

In [24]:
%%sql
DROP TABLE IF EXISTS mst_table, mst_table_summary;

SELECT madlib.load_model_selection_table('model_arch_library', -- model architecture table
                                         'mst_table',          -- model selection table output
                                          ARRAY[1,2],              -- model ids from model architecture table
                                          ARRAY[                   -- compile params
                                              $$loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']$$,
                                              $$loss='categorical_crossentropy', optimizer='Adam(lr=0.01)',metrics=['accuracy']$$,
                                              $$loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']$$
                                          ],
                                          ARRAY[                    -- fit params
                                              $$batch_size=4,epochs=1$$,
                                              $$batch_size=8,epochs=1$$
                                          ]
                                         );
                                  
SELECT * FROM mst_table ORDER BY mst_key;

Done.
1 rows affected.
12 rows affected.


mst_key,model_id,compile_params,fit_params
1,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=4,epochs=1"
2,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=8,epochs=1"
3,1,"loss='categorical_crossentropy', optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=4,epochs=1"
4,1,"loss='categorical_crossentropy', optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=8,epochs=1"
5,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']","batch_size=4,epochs=1"
6,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']","batch_size=8,epochs=1"
7,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=4,epochs=1"
8,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=8,epochs=1"
9,2,"loss='categorical_crossentropy', optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=4,epochs=1"
10,2,"loss='categorical_crossentropy', optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=8,epochs=1"


The name of the model architecture table is stored in the summary table:

In [25]:
%%sql
SELECT * FROM mst_table_summary;

1 rows affected.


model_arch_table
model_arch_library


<a id="load_model_selection_manual"></a>
# 4.  Create model selection table manually

If you would like to have more control over the set of model selection parameters to run, you can manually create the model selection table and the associated summary table.  Both must be created since they are needed by the multiple model fit module.

For example, let's say we don't want all combinations but only want batch_size=4 for model_id=1 and batch_size=8 for model_id=2:

In [26]:
%%sql
DROP TABLE IF EXISTS mst_table_manual;

CREATE TABLE mst_table_manual(
    mst_key serial,
    model_arch_id integer,
    compile_params varchar,
    fit_params varchar
);

INSERT INTO mst_table_manual(model_arch_id, compile_params, fit_params) VALUES
(1, $$loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']$$, 'batch_size=4,epochs=1'),
(1, $$loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']$$, 'batch_size=4,epochs=1'),
(1, $$loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']$$, 'batch_size=4,epochs=1'),
(2, $$loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']$$, 'batch_size=8,epochs=1'),
(2, $$loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']$$, 'batch_size=8,epochs=1'),
(2, $$loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']$$, 'batch_size=8,epochs=1');

SELECT * FROM mst_table_manual ORDER BY mst_key; 

Done.
Done.
6 rows affected.
6 rows affected.


mst_key,model_arch_id,compile_params,fit_params
1,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=4,epochs=1"
2,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=4,epochs=1"
3,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']","batch_size=4,epochs=1"
4,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=8,epochs=1"
5,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=8,epochs=1"
6,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']","batch_size=8,epochs=1"


Create the summary table which must be named with the model selection output table appended by "_summary":

In [27]:
%%sql
DROP TABLE IF EXISTS mst_table_manual_summary;

CREATE TABLE mst_table_manual_summary (
    model_arch_table varchar
);

INSERT INTO mst_table_manual_summary(model_arch_table) VALUES
('model_arch_library');

SELECT * FROM mst_table_manual_summary; 

Done.
Done.
1 rows affected.
1 rows affected.


model_arch_table
model_arch_library


<a id="load_model_selection_auto"></a>
# 5. Generate hyperparameters automatically

You can use other libraries or methods to generate hyperparameters according to the tests that you want to run.  For example, let's randomly generate batch size from powers of 2 and learning rate on a log scale.

We use psycopg which is a PostgreSQL database adapter for the Python programming language.

In [28]:
import numpy as np
import psycopg2 as p2
conn = p2.connect('postgresql://gpadmin@localhost:8000/madlib')
#conn = p2.connect('postgresql://fmcquillan@localhost:5432/madlib')
cur = conn.cursor()

%sql DROP TABLE IF EXISTS mst_table_auto, mst_table_auto_summary;

#compile params
learning_rate = np.random.permutation([0.1,0.01,0.001,0.0001])[:3]
compile_param1 = "loss='categorical_crossentropy',optimizer='Adam(lr=" + str(learning_rate[0]) + ")',metrics=['accuracy']"
compile_param2 = "loss='categorical_crossentropy',optimizer='Adam(lr=" + str(learning_rate[1]) + ")',metrics=['accuracy']"
compile_param3 = "loss='categorical_crossentropy',optimizer='Adam(lr=" + str(learning_rate[2]) + ")',metrics=['accuracy']"
compile_params = [compile_param1,compile_param2,compile_param3]

#fit params
batch_size = np.random.permutation([4,8,16,32,64])[:2]
fit_param1 = "batch_size=" + str(batch_size[0]) + ",epochs=1"
fit_param2 = "batch_size=" + str(batch_size[1]) + ",epochs=1"
fit_params = [fit_param1,fit_param2]

query = "SELECT madlib.load_model_selection_table('model_arch_library', 'mst_table_auto', ARRAY[1,2], %s, %s);"

cur.execute(query,[compile_params, fit_params])
conn.commit()

#review model selection table
%sql SELECT * FROM mst_table_auto ORDER BY mst_key;

Done.
12 rows affected.


mst_key,model_id,compile_params,fit_params
1,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=32,epochs=1"
2,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=64,epochs=1"
3,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=32,epochs=1"
4,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=64,epochs=1"
5,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']","batch_size=32,epochs=1"
6,1,"loss='categorical_crossentropy',optimizer='Adam(lr=0.001)',metrics=['accuracy']","batch_size=64,epochs=1"
7,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=32,epochs=1"
8,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.1)',metrics=['accuracy']","batch_size=64,epochs=1"
9,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=32,epochs=1"
10,2,"loss='categorical_crossentropy',optimizer='Adam(lr=0.01)',metrics=['accuracy']","batch_size=64,epochs=1"


The name of the model architecture table is stored in the summary table:

In [29]:
%%sql
SELECT * FROM mst_table_auto_summary;

1 rows affected.


model_arch_table
model_arch_library
